# Example case of estimating observing time for assumed A10 clusters #

This thread shows how to use _M2_ProposalTools_ to estimate the required time to observe clusters based on a single pointing.

In [1]:
import M2_ProposalTools.WorkHorse as WH
import numpy as np
import astropy.units as u
import M2_ProposalTools.FilterImages as FI
import scipy

Let's just take the case of a few masses at high redshift ($z=1.0$)

In [2]:
M500s = np.array([3.0,4.0,5.0,6.0])*1e14*u.M_sun
lgM200   = 15.1                    # Log_10 M200
M200     = np.power(10,15.1)       # Now in normal units
###How to go to M500?
M500s     = np.array([0.7]) * M200*u.M_sun 
zs    = np.array([0.411]) 

In [3]:
h70            = 1                          # Normalization of Hubble parameter
fwhm           = 10.0                       # Roughly the resolution of MUSTANG-2
pixsize        = 2.0                        # Assume this pixel size
s2f            = np.sqrt(8.0*np.log(2.0))   # Conversion between FWHM and sigma
pix_sigma      = fwhm/(pixsize*s2f)         # Gaussian sigma, in pixel size
scansize       = 3.0                        # Assume scan sizes of 3 arcminutes
y2k            = -3.3                       # Approximate conversion.

We'll just dive into some of the functions and get what we want: 

In [4]:
TotalTime=0
for M500,z in zip(M500s,zs):

    y500       = WH.y_delta_from_mdelta(M500.value,z,delta=500,YMrel="A10",ycyl=False,h70=h70)
    
    ymap       = WH.make_A10Map(M500,z,pixsize=pixsize,Dist=True)
    mymap      = WH.smooth_by_M2_beam(ymap,pixsize=pixsize)
    
    tab        = WH.get_xfertab(scansize)
    yxfer_init = FI.apply_xfer(mymap,tab,pixsize)
    yxfer      = scipy.ndimage.filters.gaussian_filter(yxfer_init, pix_sigma*0.9)
    uKmap      = yxfer*y2k*1e6
    uJymap     = uKmap * (56/73.0)
    MS         = 56.2
    
    #print("pixel size: ",pixsize)
    SZpeak_uK  = np.min(uKmap)
    SZpeak_uJy = np.min(uJymap)
    SZpeak_yr  = np.max(ymap)
    SZpeak_yc  = np.max(mymap)
    SZpeak_yx  = np.max(yxfer)
    ObsTime    = (MS*10/SZpeak_uJy)**2
    TelTime    = np.ceil(ObsTime*8)/4.0
    print("M500 (1e14): ",(M500/1e14).value)
    print("Redshift: ",z)
    #print("R500 (arcmin): ",Theta500*60*180/np.pi)
    print("Y500 (1e-12 sr): ",y500*1e12)
    print("Unfiltered y peak: ", SZpeak_yr)
    print("Beam-convolved y peak: ", SZpeak_yc)
    print("Filtered y peak: ", SZpeak_yx)
    print("Filtered peak, uK: ", SZpeak_uK)
    print("Filtered peak, uJy/beam: ", SZpeak_uJy)
    print("Time to 10 sigma peak detection (hrs): ",ObsTime)
    print("================================================")
    TotalTime += TelTime
    
print("Total Telescope Time: ", TotalTime)

M500 (1e14):  8.812477882559163
Redshift:  0.411
Y500 (1e12 sr):  111.41646803767371
Unfiltered y peak:  0.0001712578226715335
Beam-convolved y peak:  0.00016522446964474616
Filtered y peak:  5.078609894123501e-05
Filtered peak, uK:  -167.59412650607553
Filtered peak, uJy/beam:  -128.56535731972917
Time to 10 sigma peak detection (hrs):  19.108416934318672
Total Telescope Time:  38.25
